### Approach-3 Hybrid Model
Here I used CNN as feature extractor and then used xgb for prediction.

In [31]:
# import required libraries
import pathlib
import yaml
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, Input 
from src.data.make_dataset import make_dataset
from xgboost import XGBRegressor
from src.data.inverse_data import inverse_dataset
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error

In [32]:
# Define the model
model = models.Sequential()
model.add(Input(shape=(50, 50, 1)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2), padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())

# summary of model
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 22, 22, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 9, 9, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 5184)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,744 (217.75 KB)

 Trainable params: 55,744 (217.75 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# generate dataset using inverse technique
curr_dir = pathlib.Path.cwd()
home_dir = curr_dir.parent.as_posix()
params = yaml.safe_load(open(f"{home_dir}/params.yaml"))["make_dataset"]
x_train, y_train, x_test, y_test, _, _ = inverse_dataset(
    params["train_size"],
    params["test_size"],
    params["val_size"],
    params["image_size"],
)

# feature extraction using cnn architecture
flatten_xtrain = model(x_train)
flatten_xtest = model(x_test)

In [34]:
# train multi-output regressor xgb
xgb = XGBRegressor(objective='reg:squarederror', max_depth=6, eta=0.1, subsample=0.8, colsample_bytree=0.8, seed=42)
multi_output_model = MultiOutputRegressor(xgb)
multi_output_model.fit(flatten_xtrain, y_train)

# prediction on test set
predictions = multi_output_model.predict(flatten_xtest)

# combined predictions
print(predictions)

[[14.423382  22.08529  ]
 [ 4.8868175 32.888165 ]
 [16.96864   10.583119 ]
 [28.742617  37.950005 ]
 [18.74762   23.017103 ]
 [15.954057   6.758513 ]
 [38.68606   34.270306 ]
 [40.904434  35.672592 ]
 [23.528002  33.528965 ]
 [ 9.974583  40.54431  ]
 [ 8.831383   7.81551  ]
 [22.597878  32.856945 ]
 [ 9.382557  12.772579 ]
 [18.161882  10.989591 ]
 [35.599068  22.380322 ]
 [48.739117  20.34075  ]
 [ 5.070031  14.693311 ]
 [35.33328   43.223137 ]
 [14.976262  38.162247 ]
 [34.22899   39.0097   ]
 [ 3.374335  40.164467 ]
 [14.267354  18.975409 ]
 [32.76317   28.968842 ]
 [43.92806   10.472413 ]
 [33.84286   21.908348 ]
 [32.444866  10.892382 ]
 [38.970467  17.786392 ]
 [ 9.433578  24.597055 ]
 [33.19523   37.353085 ]
 [ 7.2226467 34.24067  ]
 [36.458595  30.124985 ]
 [ 7.12547   13.101719 ]
 [43.059643   8.437419 ]
 [13.951234  11.209703 ]
 [20.130384  37.261765 ]
 [30.473618  33.475082 ]
 [21.78665   47.523506 ]
 [11.874292  41.094273 ]
 [47.662743  43.35465  ]
 [26.37993   22.127502 ]


In [35]:
# Calculate mae for each co-ordinate
mae_x = mean_absolute_error(y_test[:, 0], predictions[:, 0])
mae_y = mean_absolute_error(y_test[:, 1], predictions[:, 1])

# Calculate average MAE (for multi-output)
avg_mae = (mae_x + mae_y) / 2

print('MAE X Coordinate: ', mae_x)
print('MAE Y Coordinate: ', mae_y)
print('Average MAE: ', avg_mae)

MAE X Coordinate:  0.9535602
MAE Y Coordinate:  1.0561363
Average MAE:  1.0048482418060303
